__从flowus导出的数据转换为称重表的格式(.csv)__

In [1]:
csv_filename = r"D:\Onedrive\ACGN相关\切煤代购\夜宵称重表\国内21.csv"

In [2]:
import csv
csv_OutputPath = csv_filename.replace(".csv", "_output.csv")

# 打开CSV文件
with open(csv_filename, newline='', encoding='utf-8') as csvfile:
    # 创建CSV阅读器
    csvreader = csv.reader(csvfile)
    # 逐行读取CSV文件
    for row in csvreader:
        # 将每行的内容转换为列表
        row_list = [item for item in row]
        m_num = row_list[0]
        try:
            gram = row_list[3]
        except:
            gram=0
        cn_list = []
        cn_list.append(row_list[1])
        if row_list[2] != '':
            cn_add_list = row_list[2].split('，')
            cn_list.extend(cn_add_list)
        # 写入新csv中
        with open(csv_OutputPath, 'a', newline='', encoding='utf-8-sig') as csvfile:
            # 创建CSV写入器
            csvwriter = csv.writer(csvfile)
            # 写入m_num和gram的值
            csvwriter.writerow([m_num, gram])
            # 写入cn_list中的每个元素
            for cn in cn_list:
                csvwriter.writerow([cn])  # 每个元素写入新的一行
            csvwriter.writerow(['',])

In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Font

# 读取CSV文件
df = pd.read_csv(csv_OutputPath)

excel_path = csv_OutputPath.replace("_output.csv", "_空白.xlsx")

# 将DataFrame写入Excel文件
df.to_excel(excel_path, index=False)


#调整格式
wb = load_workbook(excel_path)
ws = wb.active
ws.column_dimensions[get_column_letter(1)].width = 14
ws.column_dimensions[get_column_letter(2)].width = 8.38
# 描边
border_style = Border(
    left=Side(border_style='thin'),
    right=Side(border_style='thin'),
    top=Side(border_style='thin'),
    bottom=Side(border_style='thin')
)
# 字体
font_style = Font(
    name='宋体',
    size=11,
    bold=False,
    italic=False,
    vertAlign='none',
    underline='none',
    strike=False,
    color="000000"
)
row = 1
empty_block = 0
while empty_block < 2:
    col = 1
    if ws.cell(row=row, column=col).value is None:
        empty_block += 1
        if empty_block == 2:
            wb.save(excel_path)
        row += 1
        continue
    else:
        ws.cell(row=row, column=col).border = border_style
        ws.cell(row=row, column=col).alignment = Alignment(
            horizontal='center', vertical='center')
        ws.cell(row=row, column=col+1).border = border_style
        ws.cell(row=row, column=col).font = font_style
        ws.cell(row=row, column=col+1).font = font_style
        empty_block = 0
        row += 1
wb.save(excel_path)

__格式化csv转换而成的excel__

In [ ]:
excel_path = "D:\Onedrive\ACGN相关\切煤代购\夜宵称重表\国内17_output_output.xlsx"

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Alignment
from openpyxl.worksheet.page import PageMargins


# 初始列号为A和B列
current_col = 1


def cut_data_to_new_columns(ws, start_col, check_row):  # 定义一个函数来剪切数据到新列
    # 检查第58行和第60行是否有数据
    if ws.cell(row=check_row - 2, column=start_col).value is None and ws.cell(row=check_row, column=start_col).value is not None:
        # 剪切起始行
        cut_from_row = 59
    else:
        cut_from_row = 60

    # 计算目标列的列号
    target_col = start_col+3  # 找到当前最大列的下一条列

    # 剪切数据
    for row in range(cut_from_row, ws.max_row + 1):
        for col in range(start_col, start_col + 2):  # 只剪切A和B列
            # 读取原位置的值
            value = ws.cell(row=row, column=col).value
            # 写入新位置
            ws.cell(row=row-cut_from_row+1, column=col+3).value = value
            # 清空原位置
            ws.cell(row=row, column=col).value = None

    # 修改列宽
    ws.column_dimensions[get_column_letter(start_col)].width = 14
    ws.column_dimensions[get_column_letter(start_col+1)].width = 8.38
    ws.column_dimensions[get_column_letter(start_col+2)].width = 6

    # 描边
    border_style = Border(
        left=Side(border_style='thin'),
        right=Side(border_style='thin'),
        top=Side(border_style='thin'),
        bottom=Side(border_style='thin')
    )
    for row in range(1, 60):
        for col in range(start_col, start_col+1):
            if ws.cell(row=row, column=col).value is not None:
                ws.cell(row=row, column=col).border = border_style
                ws.cell(row=row, column=col).alignment = Alignment(
                    horizontal='center', vertical='center')
                ws.cell(row=row, column=col+1).border = border_style

    # 更新工作表的最大列
    return target_col


# 循环直到没有数据需要剪切
while True:
    wb = load_workbook(filename=excel_path)
    ws = wb.active

    # 检查A列的第60行是否有数据
    if ws.cell(row=60, column=current_col).value is None and ws.cell(row=61, column=current_col).value is None:
        # 这里还需要一段将当前目标列的数据格式化的代码

        break

    # 执行剪切操作
    current_col = cut_data_to_new_columns(ws, current_col, 60)
    wb.save(excel_path)


# 修改页面，设置页边距
wb = load_workbook(filename=excel_path)
ws = wb.active
margins = PageMargins()
margins.bottom = 0.75  # 下边距，单位为英寸
margins.top = 0.75     # 上边距
margins.left = 0.7    # 左边距
margins.right = 0.7   # 右边距
ws.page_margins = margins
wb.save(excel_path)